<a href="https://colab.research.google.com/github/Juvacampos/dio-lab-open-source/blob/main/C%C3%B3pia_de_transfer_learning_DIO4_JuCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

In [ ]:
# Carregar o dataset Cats vs Dogs
dataset_name = "cats_vs_dogs"
dataset, info = tfds.load(dataset_name, as_supervised=True, with_info=True)

# Separar treino e teste
train_ds = dataset['train'].take(20000)
test_ds = dataset['train'].skip(20000)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.HWTXVL_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [ ]:
# Função de pré-processamento
def preprocess(image, label):
    image = tf.image.resize(image, (160, 160))  # Ajustar tamanho para MobileNetV2
    image = image / 255.0  # Normalizar
    return image, label

In [ ]:
# Aplicar pré-processamento
train_ds = train_ds.map(preprocess).batch(32).shuffle(1000)
test_ds = test_ds.map(preprocess).batch(32)

# Carregar modelo pré-treinado MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(160, 160, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False  # Congelar pesos do modelo pré-treinado

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Construir a nova cabeça da rede
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Treinar o modelo
history = model.fit(train_ds, validation_data=test_ds, epochs=5)

# Avaliação final
test_loss, test_acc = model.evaluate(test_ds)
print(f"Acurácia no teste: {test_acc:.4f}")

# Salvar o modelo
model.save('cats_vs_dogs_transfer_learning.keras')

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 92s 84ms/step - accuracy: 0.9550 - loss: 0.1127 - val_accuracy: 0.9767 - val_loss: 0.0631
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 110s 50ms/step - accuracy: 0.9812 - loss: 0.0508 - val_accuracy: 0.9721 - val_loss: 0.0690
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 75s 82ms/step - accuracy: 0.9849 - loss: 0.0383 - val_accuracy: 0.9798 - val_loss: 0.0539
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 62s 53ms/step - accuracy: 0.9885 - loss: 0.0316 - val_accuracy: 0.9758 - val_loss: 0.0600
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 56s 53ms/step - accuracy: 0.9899 - loss: 0.0280 - val_accuracy: 0.9788 - val_loss: 0.0595
102/102 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - accuracy: 0.9790 - loss: 0.0577
Acurácia no teste: 0.9788
